<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/Code_Interpreter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Code Interpreter
Beispiel für die Verwendung der OpenAI Assistant API und des Code Interpreter-Features.

## Setup

In [18]:
!pip install -qU openai

## Assistant erstellen

In [20]:
import openai
import os
import time

# OpenAI API Key aus Colab Secret auslesen und OpenAI Client erstellen
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
client = openai.OpenAI(api_key=api_key)

assistant = client.beta.assistants.create(
  name="Technical Assistant",
  instructions="You helps people with technical issues.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

## Thread erstellen

In [ ]:
thread = client.beta.threads.create()

## Anweisung absetzen

In [ ]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="create a qr-code for 'www.neogeeks.de' and give it back to me."
)

## Run und Event Handler erstellen

In [ ]:
from typing_extensions import override
from openai import AssistantEventHandler

class EventHandler(AssistantEventHandler):
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)

  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)

  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)

  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)


with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


##Mit Dateien arbeiten

In [ ]:
# Datei hochladen
file_path = "/content/Umsatz.csv"  # Pfad zur hochzuladenden Datei
with open(file_path, "rb") as file:
    uploaded_file = client.files.create(file=file, purpose="assistants")

# Assistenten erstellen
assistant = client.beta.assistants.create(
    name="Data Analyst",
    instructions="Du bist ein Assistent, der Daten analysiert und bewertet.",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [uploaded_file.id]
      }
    },
    model="gpt-4-1106-preview",
)

# Neuen Thread erstellen
thread = client.beta.threads.create()

# Benutzeranfrage mit Datei-ID hinzufügen
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Bitte ermittle den durchschnittlichen Umsatz in der hochgeladenen Datei.",
)

# Ausführung starten
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# Auf Abschluss der Ausführung warten
while run.status in ["queued", "in_progress"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Ergebnisse abrufen
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in messages.data:
        if msg.role == "assistant":
            for content in msg.content:
                if content.type == "text":
                    print(content.text.value)
                elif content.type == "code":
                    print("Generierter Code:\n", content.code.value)
                elif content.type == "image_file":
                    print("Generiertes Bild:", content.image_file.file_id)
else:
    print("Die Ausführung ist fehlgeschlagen.")




## OCR anwenden

In [ ]:
# Datei hochladen
file_path = "/content/R240004.png"  # Pfad zur hochzuladenden Datei
with open(file_path, "rb") as file:
    invoice_file = client.files.create(file=file, purpose="assistants")

# Assistenten erstellen
assistant = client.beta.assistants.create(
    name="Data Analyst",
    instructions="Du bist ein Assistent, der Daten analysiert und bewertet.",
    tools=[{"type": "code_interpreter"}],
    tool_resources={
      "code_interpreter": {
        "file_ids": [invoice_file.id]
      }
    },
    model="gpt-4-1106-preview",
)

# Neuen Thread erstellen
thread = client.beta.threads.create()

# Benutzeranfrage mit Datei-ID hinzufügen
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Bitte ermittle die Rechnungsnummer in der hochgeladenen Datei.",
)

# Ausführung starten
run = client.beta.threads.runs.create(thread_id=thread.id, assistant_id=assistant.id)

# Auf Abschluss der Ausführung warten
while run.status in ["queued", "in_progress"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Ergebnisse abrufen
if run.status == "completed":
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    for msg in messages.data:
        if msg.role == "assistant":
            for content in msg.content:
                if content.type == "text":
                    print(content.text.value)
                elif content.type == "code":
                    print("Generierter Code:\n", content.code.value)
                elif content.type == "image_file":
                    print("Generiertes Bild:", content.image_file.file_id)
else:
    print("Die Ausführung ist fehlgeschlagen.")